# Compare model outputs of AICORE DL Model


### Imports 

In [4]:
import pandas as pd
import os
import glob
from pathlib import Path
import yaml
import datetime

# workaround buggy autocomplete
%config Completer.use_jedi = False

### Functions 

In [5]:
def get_props_from_config(config_file):
    config = yaml.load(open(config_file), Loader=yaml.BaseLoader)
    m = config['model']
    try:
        resume = config['resume'][1]
    except:
        resume = ''
    return config, resume

In [6]:
def get_vals_from_dir(subdir):
    basename = subdir.name
    
    df_tr = pd.read_csv(subdir / 'train.csv')
    df_tr['basename'] = basename[:]
    df_tr['type'] = 'train'
    df_tr['region'] = basename.split('__')[2]
    #df_list_tr.append(df_tr)
    
    
    df_v = pd.read_csv(s / 'val.csv')
    df_v['basename'] = basename[:]
    df_v['type'] = 'val'
    df_v['region'] = basename.split('__')[2]
    #df_list_v.append(df_v)
    
    return df_tr, df_v

### Settings 

need to open dir in explorer to connect !!!

In [7]:
LOGDIR = Path(r'.\logs')
#LOGDIR = Path(r'L:\initze\aicore-uc2_augmentation\logs')
#SUB_REGEX = '*'
SUB_REGEX = '*CV6*'

In [8]:
subdirs = list(LOGDIR.glob(SUB_REGEX))

### Data Loading 
- add config reader

In [20]:
df_list_tr = []
df_list_v = []
for s in subdirs:
    try:
        df_tr, df_v = get_vals_from_dir(s)
    except:
        continue
    config_file = s / 'config.yml'

    config, resume = get_props_from_config(config_file)

    for df in [df_tr, df_v]:
        df['architecture'] = config['model']['architecture']
        df['backbone'] = config['model']['encoder']
        df['loss_function'] = config['loss_function']
        df['a_b'] = df['architecture'] + '_' + df['backbone']
        df['resume'] = resume
        df['stack_height'] = int(config['model_args']['stack_height'])
        df['timestamp'] = datetime.datetime.strptime(config['run_info']['timestamp'], '%Y-%m-%d_%H-%M-%S')
        df['data_sources'] = df.apply(lambda x: config['data_sources'], axis=1)
        df['PR_diff'] = df['Precision'] - df['Recall']
    
    df_list_tr.append(df_tr)
    df_list_v.append(df_v)

df_train = pd.concat(df_list_tr).reset_index()
df_val = pd.concat(df_list_v).reset_index()

In [21]:
len(df_val)

60

#### make proper datatypes 

#### Filter by date
* only load runs later than March 22

In [22]:
date = datetime.datetime(2021,3,22)
df_train = df_train[df_train.timestamp > date]
df_val = df_val[df_val.timestamp > date]

In [23]:
len(df_val)

60

#### Merge resumed trainings

#### Filter
1. remove incomplete
2. remove stack_height = 5

In [24]:
grouped = df_val.groupby('basename').count()
bn = grouped[grouped.Epoch >= 20].index
df_val = df_val[df_val['basename'].isin(bn)]
df_train = df_train[df_train['basename'].isin(bn)]

In [25]:
df_val = df_val[df_val.stack_height == 6]
df_train = df_train[df_train.stack_height == 6]

### Data Export

In [27]:
pd.to_pickle(df_val, 'df_val_filtered_add20.pkl')

In [28]:
pd.to_pickle(df_train, 'df_train_filtered_add20.pkl')